In [9]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD,Adam
import numpy as np
import matplotlib.pyplot as plt
from utils import WeightReader, decode_netout, draw_boxes, normalize

In [10]:
batch_size = 128
num_classes = 80
epochs = 5

In [11]:
#perform data prep

In [16]:
model = Sequential()

model.add(keras.layers.BatchNormalization(input_shape=(448,448,3), name="norm_1"))
model.add(Conv2D(16, (3,3), input_shape=(448,448,3), strides=1, name="conv_1"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_2"))
model.add(Conv2D(32, (3,3), strides=1, name="conv_2"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_3"))
model.add(Conv2D(64, (3,3), strides=1, name="conv_3"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_4"))
model.add(Conv2D(128, (3,3), strides=1, name="conv_4"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_5"))
model.add(Conv2D(256, (3,3), strides=1, name="conv_5"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_6"))
model.add(Conv2D(512, (3,3), strides=1, name="conv_6"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization(name="norm_7"))
model.add(Conv2D(1024, (3,3), strides=1, name="conv_7"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(keras.layers.BatchNormalization(name="norm_8"))
model.add(Conv2D(1024, (3,3), strides=1, name="conv_8"))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(keras.layers.BatchNormalization(name="norm_9"))
model.add(Conv2D(425, (1,1), strides=1, activation = "linear", name="conv_9"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
norm_1 (BatchNormalization)  (None, 448, 448, 3)       12        
_________________________________________________________________
conv_1 (Conv2D)              (None, 446, 446, 16)      448       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 446, 446, 16)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 223, 223, 16)      0         
_________________________________________________________________
norm_2 (BatchNormalization)  (None, 223, 223, 16)      64        
_________________________________________________________________
conv_2 (Conv2D)              (None, 221, 221, 32)      4640      
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 221, 221, 32)      0         
__________

In [17]:
wt_path = 'tinyyolo.weights'
weight_reader = WeightReader(wt_path)
weight_reader.reset()
nb_conv = 9

for i in range(1, nb_conv+1):
    conv_layer = model.get_layer('conv_' + str(i))
    
    if i < nb_conv:
        norm_layer = model.get_layer('norm_' + str(i))
        
        size = np.prod(norm_layer.get_weights()[0].shape)

        beta  = weight_reader.read_bytes(size)
        gamma = weight_reader.read_bytes(size)
        mean  = weight_reader.read_bytes(size)
        var   = weight_reader.read_bytes(size)

        weights = norm_layer.set_weights([gamma, beta, mean, var])       
        
    if len(conv_layer.get_weights()) > 1:
        bias   = weight_reader.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
        kernel = weight_reader.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
        kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
        kernel = kernel.transpose([2,3,1,0])
        conv_layer.set_weights([kernel, bias])
    else:
        kernel = weight_reader.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
        kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
        kernel = kernel.transpose([2,3,1,0])
        conv_layer.set_weights([kernel])

ValueError: cannot reshape array of size 128728 into shape (425,1024,1,1)